In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y/50
    

In [3]:
label = "1D_SODE_atanh_scaled"
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0/50.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(3.0)
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100

N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    alpha_val = []
    
    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 
    
    
    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.5, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')
   

    start_time = time.time()
    train_model(max_iter,reps)

    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    alpha_full.append(alpha_val)    
    
    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "alpha": alpha_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 28932.719 Test MSE 1.2513083475211464 Test RE 2.8511151495293943
1 Train Loss 6420.2207 Test MSE 1.388448494353748 Test RE 3.0032914091182286
2 Train Loss 2729.6077 Test MSE 1.597508223969185 Test RE 3.22147034738217
3 Train Loss 1014.9164 Test MSE 1.6163467122108437 Test RE 3.240409143

96 Train Loss 0.0036896989 Test MSE 0.42424938605625095 Test RE 1.660134206377826
97 Train Loss 0.0036896986 Test MSE 0.42424938605625095 Test RE 1.660134206377826
98 Train Loss 0.0036896986 Test MSE 0.42424938605625095 Test RE 1.660134206377826
99 Train Loss 0.0036896989 Test MSE 0.42424938605625095 Test RE 1.660134206377826
Training time: 26.26
Training time: 26.26
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=Tru

90 Train Loss 0.016998293 Test MSE 0.17716173647974467 Test RE 1.0727970017653148
91 Train Loss 0.016978102 Test MSE 0.17704476636156305 Test RE 1.0724427889591164
92 Train Loss 0.01695951 Test MSE 0.17693995611827545 Test RE 1.072125299680951
93 Train Loss 0.01693431 Test MSE 0.1768573568642182 Test RE 1.0718750252675309
94 Train Loss 0.016901338 Test MSE 0.1767856850396395 Test RE 1.0716578134282368
95 Train Loss 0.016862672 Test MSE 0.17675392940452106 Test RE 1.0715615593031882
96 Train Loss 0.016790839 Test MSE 0.1767369119106614 Test RE 1.0715099742231506
97 Train Loss 0.016684042 Test MSE 0.1768167579200396 Test RE 1.0717519896958438
98 Train Loss 0.016570464 Test MSE 0.17692646070681742 Test RE 1.07208441279404
99 Train Loss 0.0147988405 Test MSE 0.17755787549880112 Test RE 1.0739957353642515
Training time: 31.85
Training time: 31.85
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias

83 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
84 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
85 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
86 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
87 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
88 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
89 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
90 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
91 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
92 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
93 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
94 Train Loss 0.0007250265 Test MSE 0.11107988656552997 Test RE 0.8494744103143022
95 T

77 Train Loss 0.23286995 Test MSE 0.12068297153917987 Test RE 0.8854327561101624
78 Train Loss 0.23040602 Test MSE 0.12074114176599174 Test RE 0.8856461234913511
79 Train Loss 0.22825912 Test MSE 0.12112099103299367 Test RE 0.8870381422136981
80 Train Loss 0.22040558 Test MSE 0.12297097653692676 Test RE 0.8937867204767648
81 Train Loss 0.21775728 Test MSE 0.12302276637170902 Test RE 0.8939749120126711
82 Train Loss 0.2169437 Test MSE 0.12267371507059847 Test RE 0.8927057779768063
83 Train Loss 0.21601675 Test MSE 0.12212388092842756 Test RE 0.8907029391930602
84 Train Loss 0.21530592 Test MSE 0.12161298347471027 Test RE 0.8888378871464068
85 Train Loss 0.2145296 Test MSE 0.12104641806162734 Test RE 0.8867650299547077
86 Train Loss 0.21400307 Test MSE 0.12069236187936834 Test RE 0.8854672031947679
87 Train Loss 0.21301195 Test MSE 0.12024272407668139 Test RE 0.8838162659290296
88 Train Loss 0.21174538 Test MSE 0.11992325528898634 Test RE 0.8826413944122776
89 Train Loss 0.21061012 Test 

72 Train Loss 0.6015011 Test MSE 0.14033819934443376 Test RE 0.9548178662690137
73 Train Loss 0.41047293 Test MSE 0.13667762079274787 Test RE 0.9422828612007418
74 Train Loss 0.27955735 Test MSE 0.16300504738623423 Test RE 1.0290420292465015
75 Train Loss 0.1379478 Test MSE 0.17683205423461923 Test RE 1.0717983470129846
76 Train Loss 0.053995747 Test MSE 0.19437319913503656 Test RE 1.1237010280584854
77 Train Loss 0.040280335 Test MSE 0.19764145563266486 Test RE 1.1331087905439667
78 Train Loss 0.02913329 Test MSE 0.19632919569685922 Test RE 1.1293408319141627
79 Train Loss 0.025015425 Test MSE 0.19059012397585826 Test RE 1.1127120293056005
80 Train Loss 0.022777498 Test MSE 0.1896273654713434 Test RE 1.109898060648579
81 Train Loss 0.020668076 Test MSE 0.1823887015990146 Test RE 1.0885078181765995
82 Train Loss 0.020081416 Test MSE 0.18253795837124082 Test RE 1.0889531142314808
83 Train Loss 0.016859675 Test MSE 0.18083855670008667 Test RE 1.083872264044176
84 Train Loss 0.013778336 T

68 Train Loss 67.783035 Test MSE 2.0096194034094395 Test RE 3.6131791422223154
69 Train Loss 66.715744 Test MSE 2.001656360827698 Test RE 3.606013492392786
70 Train Loss 66.19269 Test MSE 2.0421346464811823 Test RE 3.6422921148961236
71 Train Loss 66.16351 Test MSE 2.0423063496120846 Test RE 3.642445234039277
72 Train Loss 65.678764 Test MSE 2.019702115247578 Test RE 3.6222318671167706
73 Train Loss 65.5342 Test MSE 2.0254350693761567 Test RE 3.6273691033571462
74 Train Loss 64.39476 Test MSE 2.1081290926185288 Test RE 3.700677057441199
75 Train Loss 63.98143 Test MSE 2.1376571097876984 Test RE 3.7265041452337075
76 Train Loss 63.815907 Test MSE 2.176505224671433 Test RE 3.7602129768011223
77 Train Loss 63.314762 Test MSE 2.0865943821032222 Test RE 3.681727180693285
78 Train Loss 62.226837 Test MSE 2.066104990894959 Test RE 3.6636061573811762
79 Train Loss 61.840424 Test MSE 1.9939936834195404 Test RE 3.5991046607575043
80 Train Loss 61.46263 Test MSE 1.9761788774938591 Test RE 3.58299

65 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
66 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
67 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
68 Train Loss 0.0042709154 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
69 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
70 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
71 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
72 Train Loss 0.0042709154 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
73 Train Loss 0.0042709154 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
74 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
75 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
76 Train Loss 0.004270916 Test MSE 0.12811280189066487 Test RE 0.9122814656789345
77 Train Loss

KeyboardInterrupt: 

In [ ]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)